# Bigram-gram based LLM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch

## Dataset Preprocessing

In [ ]:
ds = load_dataset("MuskumPillerum/General-Knowledge")['train']

In [ ]:
l = []
for i,j in zip(ds['Question'], ds['Answer']):
    if i and j:
        i = ''.join(i.replace(".", "").replace("'", "").replace('"', "").replace('\\n', "").split('?')) + ' ?'
        j = j.replace('\\n', "")
        l.append(f'<Start> Question: {i.lower()} Answer: {j.lower()} <Stop>')

    # break

In [ ]:
len(l)

In [ ]:
vocab = []
for sentence in l[:100]:
    vocab.extend(sentence.split())

len(vocab)


In [ ]:
vocab = set(vocab)

vocab_dict = {} 
ct = 0
for i in vocab:
    vocab_dict[i] = ct
    ct += 1
print(ct)

In [ ]:
rev_vocab_dict = {v:k for k,v in vocab_dict.items()}

## Model Creation

In [ ]:
n_gram_matrix = np.zeros((len(vocab), len(vocab)), dtype=np.int8) +.1 # 15 gig!
n_gram_matrix.shape

In [ ]:
for sentence in l[:100]:
    prev_word_index = vocab_dict['<Start>']
    # print(sentence)
    for word in sentence.split()[1:]:
        index = vocab_dict[word]
        n_gram_matrix[prev_word_index][index] += 100 # So that it is bigger than 1
        prev_word_index = index
        # print(word)
        
    # break

In [ ]:
for i in n_gram_matrix:
    print(i)

In [ ]:

# print(i/i.sum())
# next_index = torch.multinomial(torch.tensor(i, dtype=torch.float16), num_samples=20, replacement=True)
# print(next_index)

# Inference

In [36]:
out = vocab_dict['<Start>']
while out != vocab_dict['<Stop>']:
    row = n_gram_matrix[out]
    # print(out)
    next_index = torch.multinomial(torch.tensor(row, dtype=torch.float16), num_samples=1, replacement=True)
    # print(next_index)
    print(rev_vocab_dict[next_index.item()], end=' ')
    out = next_index
    # break

Question: what is the cuttlefish is the longest migration among living over time. <Stop> 

In [ ]:
rev_vocab_dict[577]